In [8]:
import torch
import torch.nn as nn
import numpy as np
torch.__version__
torch.manual_seed(0)

## Linear Regression

useful links

https://www.youtube.com/watch?v=zPG4NjIkCjc

https://www.kaggle.com/aakashns/pytorch-basics-linear-regression-from-scratch

Linear regression models a linear relationship between two variables. There is usually an independent value $x$
and a dependent value $y$. Linear regression has an equation with the form $y=ax+b$ and finds the optimal values
$a$ and $b$ that best describe the relationship of the variables. More specifically, this equation describes a straight
line with slope eaual to $a$ and $b$ the intercept (the value of $y$ when $x = 0$).



Let's create and initialize randomly our model's variables  $a$ and $b$.

In [11]:

a = torch.randn(1,requires_grad=True)
b = torch.randn(1,requires_grad=True)
print(a)
print(b)


x = torch.randn(10)
y = a*x+b



tensor([0.1919], requires_grad=True)
tensor([1.2638], requires_grad=True)


Fit our model to random data.

In [18]:

x = torch.randn((10,1))
x = x*2
y = torch.randn((10,1))
y * y*2


class Linear_regression_model:
    def __init__(self):
        super(Linear_regression_model, self).__init__()

        self.a = torch.randn((1,1),requires_grad=True)
        self.b = torch.randn((1,1),requires_grad=True)
        self.lr = 0.001
    def forward(self,x):
        y_hat = torch.matmul(x , self.a.t()) + self.b
        #y_hat = x@  self.a.t() + self.b
        return y_hat
    def mse_loss(self,y,y_hat):
        diff = y-y_hat
        self.loss = (diff*diff).mean()
        
        return self.loss.item()
    def update_params(self):
        self.loss.backward()
        grad_a = - self.lr*self.a.grad
        grad_b = - self.lr*self.b.grad
        self.a = self.a +grad_a
        self.b = self.b +grad_b
        if self.a.grad is not None:
            self.a.grad.zero_()
        if self.b.grad is not None:
            self.b.grad.zero_()

model = Linear_regression_model()
y_hat = model.forward(x)

print(y_hat)
# y_hat.mean().backward()
# print(model.a.grad)

tensor([[-0.8456],
        [-0.6027],
        [-1.4105],
        [-0.1005],
        [-0.4007],
        [-0.5887],
        [-0.6124],
        [-0.8205],
        [-0.3867],
        [-0.2400]], grad_fn=<AddBackward0>)


MSE LOSS

In [19]:
# def mse(x1,x2):
#     return ((x1-x2)**2).mean()
#
# loss = mse(y,y_hat)
# print(loss)
#
# loss.backward()

loss = model.mse_loss(y,y_hat)
lr = 0.0001
# print(f'Before update {a}  {b} Gradients {a.grad} {b.grad}')
# a = a-lr*a.grad
# b = b-lr*b.grad
# if a.grad is not None:
#     a.grad.zero_()
# if b.grad is not None:
#     b.grad.zero_()
#
#
#
# print(f'After update {a}  {b}')
# def update_parameters(a,b,lr=0.0001):
#     a = a-lr*a.grad
#     b = b-lr*b.grad
#     b.grad.zero_()
#     a.grad.zero_()


MSE LOSS

$MSE = \frac{1}{N}

In [ ]:
# Weights and biases
a = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)

a = torch.randn(1, 1, requires_grad=True)
b = torch.randn(1, requires_grad=True)
print(a)
print(b)


In [ ]:


# Define the model
def model(x):
    return x @ a.t() + b

# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()
def mse(y,y_hat):
     return((y-y_hat)**2).mean()
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')


# Input (temp, rainfall, humidity)
inputs = 0.001*np.array([[73 ], 
                   [91 ], 
                   [87], 
                   [102], 
                   [69]], dtype='float32')

# Targets (apples, oranges)
targets = 0.001*np.array([[56], 
                    [81], 
                    [119], 
                    [22], 
                    [103]], dtype='float32')

# Convert inputs and targets to tensors

print(inputs.shape)
print(targets.shape)
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
# Generate predictions
preds = model(inputs)
print(preds)


# Compute loss
loss = mse(preds, targets)
print(loss)

# Compute gradients
loss.backward()



# Gradients for weights
print(a)
print(a.grad)



# Gradients for bias
print(b)
print(b.grad)



a.grad.zero_()
b.grad.zero_()
print(a.grad)
print(b.grad)


In [ ]:
# Train for 100 epochs
lr = 1e-3
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    print(f'Loss {loss.item():.2f}')
    loss.backward()
    with torch.no_grad():
        a -= a.grad * lr
        b -= b.grad * lr
        a.grad.zero_()
        b.grad.zero_()

In [ ]:

# model = Linear_regression_model()
# num_of_iterations = 100
# for i in range(num_of_iterations):
#     y_hat =model.forward(x)
#     loss = model.mse_loss(y,y_hat)
#     print(f'LOSS {loss}')
#     model.update_params()


x1 = torch.randn(10,1)

x = torch.tensor(x1)

y = torch.randn(10,1)



a = torch.randn(1,1,requires_grad=True)
b = torch.randn(1,requires_grad=True)
num_of_iterations = 100
for i in range(num_of_iterations):
    y_hat = x@a.t() + b
    loss = ((y-y_hat)**2).mean()
    print(f'LOSS {loss.item()}')
    loss.backward()

    a = a-lr*a.grad
    b = b-lr*b.grad
    if a.grad is not None:
        a.grad.zero_()
    if b.grad is not None:
        b.grad.zero_()

tensor([[-0.3698]], requires_grad=True)
tensor([-1.6349], requires_grad=True)
(5, 1)
(5, 1)
tensor([[-1.6619],
        [-1.6686],
        [-1.6671],
        [-1.6726],
        [-1.6604]], grad_fn=<AddBackward0>)
tensor(3.0367, grad_fn=<MeanBackward0>)
tensor([[-0.3698]], requires_grad=True)
tensor([[-0.2938]])
tensor([-1.6349], requires_grad=True)
tensor([-3.4846])
tensor([[0.]])
tensor([0.])


In [43]:
# Train for 100 epochs
lr = 1e-3
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    print(f'Loss {loss.item():.2f}')
    loss.backward()
    with torch.no_grad():
        a -= a.grad * lr
        b -= b.grad * lr
        a.grad.zero_()
        b.grad.zero_()

Loss 3.04
Loss 3.02
Loss 3.01
Loss 3.00
Loss 2.99
Loss 2.98
Loss 2.96
Loss 2.95
Loss 2.94
Loss 2.93
Loss 2.92
Loss 2.90
Loss 2.89
Loss 2.88
Loss 2.87
Loss 2.86
Loss 2.85
Loss 2.84
Loss 2.82
Loss 2.81
Loss 2.80
Loss 2.79
Loss 2.78
Loss 2.77
Loss 2.76
Loss 2.75
Loss 2.73
Loss 2.72
Loss 2.71
Loss 2.70
Loss 2.69
Loss 2.68
Loss 2.67
Loss 2.66
Loss 2.65
Loss 2.64
Loss 2.63
Loss 2.62
Loss 2.61
Loss 2.59
Loss 2.58
Loss 2.57
Loss 2.56
Loss 2.55
Loss 2.54
Loss 2.53
Loss 2.52
Loss 2.51
Loss 2.50
Loss 2.49
Loss 2.48
Loss 2.47
Loss 2.46
Loss 2.45
Loss 2.44
Loss 2.43
Loss 2.42
Loss 2.41
Loss 2.40
Loss 2.39
Loss 2.38
Loss 2.37
Loss 2.37
Loss 2.36
Loss 2.35
Loss 2.34
Loss 2.33
Loss 2.32
Loss 2.31
Loss 2.30
Loss 2.29
Loss 2.28
Loss 2.27
Loss 2.26
Loss 2.25
Loss 2.24
Loss 2.24
Loss 2.23
Loss 2.22
Loss 2.21
Loss 2.20
Loss 2.19
Loss 2.18
Loss 2.17
Loss 2.16
Loss 2.16
Loss 2.15
Loss 2.14
Loss 2.13
Loss 2.12
Loss 2.11
Loss 2.10
Loss 2.10
Loss 2.09
Loss 2.08
Loss 2.07
Loss 2.06
Loss 2.05
Loss 2.05
Loss 2.04


In [39]:

# model = Linear_regression_model()
# num_of_iterations = 100
# for i in range(num_of_iterations):
#     y_hat =model.forward(x)
#     loss = model.mse_loss(y,y_hat)
#     print(f'LOSS {loss}')
#     model.update_params()


x1 = torch.randn(10,1)

x = torch.tensor(x1)

y = torch.randn(10,1)



a = torch.randn(1,1,requires_grad=True)
b = torch.randn(1,requires_grad=True)
num_of_iterations = 100
for i in range(num_of_iterations):
    y_hat = x@a.t() + b
    loss = ((y-y_hat)**2).mean()
    print(f'LOSS {loss.item()}')
    loss.backward()

    a = a-lr*a.grad
    b = b-lr*b.grad
    if a.grad is not None:
        a.grad.zero_()
    if b.grad is not None:
        b.grad.zero_()

LOSS 1.2500154972076416
LOSS 1.247922658920288


/home/iliasprc/Documents/penvs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/home/iliasprc/Documents/penvs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
/home/iliasprc/Documents/penvs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .gra

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'